# Morphological analysis with FSTs
The following is a brief and basic tutorial on how to construct a **morphological analyzer** for a language using finite-state techniques. A toy grammar of English noun and verb inflections is built step-by-step to illustrate overall design issues. While the grammar is small, much larger grammars can be built using the same design principles. This tutorial uses the [Helsinki Finite-State Transducer toolkit](http://hfst.github.io/).

In [ ]:
import hfst
import fstutils as fst

In [ ]:
help(fst.remove_epsilons)

# Design
The construction of the final transducer is broken down into two large components:

- A lexicon
- Alternation rules

## Lexicon

The lexicon component is a transducer that:

- Accepts as input the valid stems/lemmas of the language, followed by a legal sequence of **tags**.
- Produces as output an intermediate form in which the tags are replaced by the morphemes that they correspond to.
- May produce additional symbols in the output, such as special symbols that serve to mark the presence of morpheme boundaries.

For example, in the analyzer to be constructed, the lexicon FST performs the following mappings:
```
c a t +N +Pl      w a t c h +N +Pl      w a t c h +V +3P +Sg     (input side)
c a t ^  s        w a t c h ^  s        w a t c h ^  s           (output side)
```

There are two things to note here:

1. We use the symbol `^` to mark a morpheme boundary.
2. While each letter in the stem is represented by its own symbol (`w`, `a`, `t`, `c`, `h`, etc.), each complete tag is a **multicharacter symbol** (`+N`, `+Pl`, etc.). The spaces in the example above show the symbol boundaries to illustrate this.

The lexicon transducer is written in a formalism called **lexc**.

## Alternation rules

The role of the alternation rules is to modify the output of the lexicon transducer according to orthographic, phonological, and morphophonological rules and conventions. So far, for example, we've assumed that English nouns can be pluralized by concatenating the morpheme *-s* to the stem (`cat` → `cats`).  However, noun stems that end in a sibilant take the allomorph *-es* (`watch` → `watches`). A way to describe the process of forming correct nouns is to always represent the plural as the morpheme *-s* and then subject these word forms to alternation rules that insert an *e* if the stem ends in a sibilant. This is one of the tasks of the rules component: to produce the valid surface forms from the intermediate forms output by the lexicon transducer.

Since rule FSTs that are conditioned by their environment are very difficult to construct by hand, we use the replace rules formalism to compile the necessary rules into FSTs.

### Replace rule basics

Replace rules are often used when modeling morphophonology.  These have the following basic format.

    LHS -> RHS || LC _ RC ;

where `LHS` is left-hand-side, `RHS` is right-hand-side, `LC` is left-context, and `RC` is right-context.

A regular expression such as:

    regex a -> b || c _ d ;

would create a transducer that converts all instances of `a` to `b`, but only if that `a` occurs between `c` and `d`.

The `LC` and `RC` are optional, so you could say:

    regex a -> 0 || _ b ;

creating a transducer that deletes `a`-symbols when they occur before a `b`.

You can also drop the context altogether, and say:

    regex x -> y ;

Which creates a transducer that maps all `x`-symbols to `y`, always, passing everything else through untouched.

#### Word boundaries in rewrite rules

The special symbol `.#.` is used to refer to edges of words.  For example, the following transducer deletes all `x`-symbols, but only at the beginning of a word.  Everything else is passed through as-is.

    regex x -> 0 || .#. _ ;

#### Epenthesis (insertion) rules

There is a special type of rule, called epenthesis rules is used whenever you want to insert something (from nothing) in a special position.  This is denoted by `[..]` as the `LHS`.  For example:

    regex [..] -> x || y _ z ;

would result in a transducer that inserts `x` between `y` and `z`.

# Lexicon

As mentioned above, the lexicon script is written in the **lexc** formalism and stored in a text file, which we call `english.lexc`.

## Features
    
We want to include the following features in the grammar.

- Nouns: singular (*cat*) vs. plural (*cats*)
- Verbs: infinitive (*watch*), 3rd person singular (*watches*), past tense (*watched*), past participle (*watched*), and present participle (*watching*)

We also want to use the following tags for marking the above grammatical information:

- `+N` for nouns
- `+V` for verbs
- `+3P` for third person
- `+Sg` for singular forms
- `+Pl` for plural forms
- `+Past` for past tense
- `+PastPart` for past participle
- `+PresPart` for present participle

## lexc

The `lexc` formalism operates under the simple notion of continuation classes. We declare a set of labeled lexicons, the content of those lexicons, and rules which dictate how lexicon entries are concatenated.

Here, we provide a line-by-line analysis of `english.lexc`.

First, we need to declare the multicharacter symbols:

```
Multichar_Symbols +N +V +PastPart +Past +PresPart +3P +Sg +Pl
```

Then, we declare a lexicon called `Root` and provide it with two entries:

```
LEXICON Root

Noun ;
Verb ;
```

Both entries are empty (i.e., there is no left-hand side), and by choosing one of these, we can move to either the `Noun` or the `Verb` lexicon.

The `Noun` lexicon looks as follows:

```
LEXICON Noun

cat   Nstem;
city  Nstem;
fox   Nstem;
panic Nstem;
try   Nstem;
watch Nstem;
```

Here, we have six entries.  All of these entries continue to the `Nstem` lexicon.

The `Verb` lexicon also has six entries, and all of them continue to the `Vinf` lexicon.

```
LEXICON Verb

beg   Vinf;
fox   Vinf;
make  Vinf;
panic Vinf;
try   Vinf;
watch Vinf;
```

The `Nstem` lexicon has two entries:

```
LEXICON Nstem

+N+Sg:0   #;
+N+Pl:^s  #;
```

Unlike in the previous lexicons, both entries specify the input string and the output string separately. For example, the entry `+N+Sg:0` specifies that the input side contains `+N+Sg` and the output side the empty string. The continuation lexicon for both is `#`, meaning end-of-word. So if we choose from the `Root` lexicon to enter the `Noun` lexicon, and from there choose the `cat` entry, and in the `Ninf` lexicon choose the `+N+Sg:0` entry, we will construct the input-output pairing:

```
c a t +N +Sg
c a t
```

In the `Vinf` lexicon, we have a few more entries, one for each verb form:

```
LEXICON Vinf

+V:0             #;
+V+3P+Sg:^s      #;
+V+Past:^ed      #;
+V+PastPart:^ed  #;
+V+PresPart:^ing #;
```

We can compile the `english.lexc` script into a transducer and label it for subsequent use.

In [ ]:
grammar = hfst.compile_lexc_file('english.lexc')

# Alternation rules

We now turn to the alternation rules component. The idea is to construct a set of ordered rule transducers that modify the intermediate forms output by the lexicon component. At the very least, we need to remove the `^` symbol which is used to separate morpheme boundaries. This is in fact the **last** rule transducer in the cascade. There are a number of phonological and orthographic rules that are needed first.

## Natural classes

Before we define the rules, we need to define the class of vowels.

In [ ]:
defs = fst.Definitions({
    "V": "[ a | i | e | o | u ]"
})

## Rules

### e-deletion

Stems that end in a silent `e` drop the `e` with certain suffixes (`ing` and `ed` in our case). For example, `make` → `making`.

In [ ]:
edeletion = hfst.regex('e -> 0 || _ "^" [ i n g | e d ]')

If we apply the rule to the output of the lexicon component, we get changes like:

```
m a k e +V +PresPart        (lexicon input)
m a k e ^  i         n g    (lexicon output)
m a k   ^  i         n g    (after e-deletion)
```

### e-insertion

This rule applies when a stem ends in a sibilant and is followed by the plural morpheme `s` (`watch` → `watches`).  Sibilants can be defined orthographically by `s`, `z`, `x`, `c h`, and `s h`.

In [ ]:
einsertion = hfst.regex('[..] -> e || [ s | z | x | c h | s h ] _ "^" s')

Note the `[..]` construction on the left-hand side. Here, using an epsilon (`0`) would insert a potentially infinite number of `e` symbols. The special construction `[..]` forces epsilon insertions to restrict themselves to one insertion per potential insertion site.

### y-replacement

This rule, which acts in constructions such as `try` → `tries` and `try` → `tried`, is perhaps best broken into two parts. Consider the corresponding input-intermediate form pairings:

```
(1)                   (2)
t r y   +V +3P +Sg    t r y +V +PastPart      (lex in)
t r y   ^  s          t r y ^  e          d   (lex out)
t r i e ^  s          t r i ^  e          d   (desired rule output)
```

Case (1) requires changing changing the `y` into `i e`, whereas case (2) requires only changing the `y` into `i`. We can accomplish this by having two parallel rules.

In [ ]:
yreplacement = hfst.regex('y -> i e || _ "^" s ,, y-> i || _ "^" e d')

Note that the `,,` separates the two rules. These rules operate simultaneously and independently. For this particular case, it doesn't matter whether the two rules apply simultaneously or one after the other.

### k-insertion

Verbs that end in a `c` add a `k` before the morpheme boundary if followed by an suffix beginning with a vowel. For example, `panic` → `panicking` `panicked`. This rule relies on us having defined V as our set of vowel symbols.

In [ ]:
kinsertion = hfst.regex(defs.replace('[..] -> k || V c _ "^" [e d | i n g]'))

### Consonant doubling (gemination)

Consonant doubling is really the same phenomenon as **k-insertion**. We double final consonants in the stem in certain environments: `beg` → `begging`, `run` → `runnable`, etc. Our lexicon contains only one such word (*beg*), so we can just use one rule. For a more thorough treatment, we'd also need to add rules for the other consonants.

In [ ]:
consonantduplication = hfst.regex('g -> g g || _ "^" [i n g | e d]')

### Auxiliary symbol removal

As the last rule, we need to remove the auxiliary `^`, whose presence is necessary for the alternation rules. The last rule removes these and produces valid surface forms.

In [ ]:
cleanup = hfst.regex('"^" -> 0')

# Compiling the grammar

We combine the lexicon FST and the various FSTs that encode alternation rules into one large transducer that acts like a cascade. This single large transducer has the same effect as providing an input to the lexicon transducer, taking its output and feeding it into the first rule transducer, taking its output and feeding it into the next rule transducer, and so on.

Normally, this cascade is accomplished by the regular expression composition operator (`.o.`). Suppose we have the lexicon transducer in an FST named `Lexicon` and the various alternation rules as FSTs named `Rule1`, ..., `RuleN`. We can issue the regular expression
```
Lexicon .o. Rule1 .o. Rule2 .o. ... .o. RuleN ;
```
and produce a single transducer that is the composite of the different rule transducers and the lexicon transducer.

However, in HFST, we compile the grammar differently.

In [ ]:
# Be careful here.
# Since composition is done in place, rerunning composes without redefining the FST from scratch will make mega-FSTs.

grammar.compose(consonantduplication)
grammar.compose(einsertion)
grammar.compose(edeletion)
grammar.compose(yreplacement)
grammar.compose(kinsertion)
grammar.compose(cleanup)

## Testing the grammar

In [ ]:
fst.lookup(grammar, 'panic+V+Past')

We can also list all the input/output pairs of the words with the `pairs` command.

In [ ]:
print(fst.pairs(grammar))